In [56]:
import mne
import braindecode
from braindecode.preprocessing.preprocess import preprocess, Preprocessor, zscore

import sys
import importlib
importlib.reload(sys.modules['braindecode.datasets.sleep_physionet'])
from braindecode.datasets.sleep_physionet import SleepPhysionet


In [59]:
from plot import Plot

p = Plot('')
p.plot_acc([''])



plotting accuracy per epoch...


KeyError: 'train_acc'

In [8]:
raw_set = [
    '/home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf',
    '/home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf',
    # '/home/maligan/mne_data/physionet-sleep-data/SC4441E0-PSG.edf',
    # '/home/maligan/mne_data/physionet-sleep-data/SC4431E0-PSG.edf',
    # '/home/maligan/mne_data/physionet-sleep-data/SC4421E0-PSG.edf',
]

In [9]:
# load into raw array
raws = [mne.io.read_raw_edf(x) for x in raw_set]

Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [6]:
# pick channels
raws = [x.pick_channels(['EEG Fpz-Cz', 'EEG Pz-Oz']) for x in raws]

In [12]:
### preprocess
# resample to 160Hz
# high pass filtering of 30Hz

sfreq = 160
high_cut_hz = 30
n_jobs = 2

for raw in raws:
    mne.io.Raw.resample(raw, sfreq, n_jobs=n_jobs) # resample all files
    mne.io.Raw.filter(raw, l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs) # high-pass filter




[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   39.5s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   40.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   42.5s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   34.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   44.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   37.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:   16.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   16.5s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   16.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   21.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   18.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   17.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:   14.8s finished


In [21]:
type(raws)

list

In [35]:
from braindecode.preprocessing import create_windows_from_events
from braindecode.datasets import BaseConcatDataset

dataset = BaseConcatDataset(raws)
dataset


In [43]:
print(f'Sampling rate before: {raws[0].info["sfreq"]}')

Sampling rate before: 160.0


In [36]:
window_size_samples = 3000

windows_dataset = create_windows_from_events(
    dataset, trial_start_offset_samples=0, trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples, preload=True)



AttributeError: 'RawEDF' object has no attribute 'raw'

In [17]:
# z-score normalization

preprocess(windows_dataset, Preprocessor(zscore))


<RawEDF | SC4451F0-PSG.edf, 7 x 13392000 (83700.0 s), ~715.2 MB, data loaded>

In [18]:
mne.io.RawArray(raws, mne.create_info())

TypeError: create_info() missing 2 required positional arguments: 'ch_names' and 'sfreq'